In [1]:
%reload_ext autoreload
%autoreload 2

! [ ! -L /datasets ] && ln -s /data/datasets/ /datasets

## 需掌握知识点

1. 人工智能的应用与价值
2. 人工智能作为一次科技浪潮的特点

In [6]:
%reset -f

from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer
import torch
from torch import nn

class CustomClassifier(EasyaiClassifier):

    ##########################################################################
    ####### Dataset ######
    ##########################################################################
    def prepare_dataset(self):
        """
        准备数据集, 从磁盘上加载数据集, 不同数据集的描述格式可能不一样, 一般有json/xml/csv等描述格式,
        也可能直接是图片目录, 所有这些格式的处理可以在这个接口完成.
        
        预置数据集: mnist, cifar10, flowers, fruits, dogcat, chestxray
            
        返回:
            以下几种方式任意一种:
            1. EasyaiDataset实例, 表明只进行训练(只返回了训练数据集实例)
            2. EasyaiDataset实例列表, 当列表长度为2时, 说明还要进行训练的校验, 当列表长度为3时, 说明还要进行测试评估.
            3. EasyaiDataset实例字典, 如: {'train': EasyaiDataset, 'val': EasyaiDataset, 'test':EasyaiDataset}
        """
        return self.load_mnist()
    
    ##########################################################################
    ####### Model ######
    ##########################################################################
    def build_model(self):
        """
        构建模型
        
        返回:
            模型实例
        """
        return self.load_resnet18(num_classes=10)
    
    ##########################################################################
    ####### Hypes Parameters ######
    ##########################################################################
    def configure_optimizer(self, model):
        """
        配置优化器
        
        返回:
            optimizer
        """
        return self.adam(model.parameters(), base_lr=0.001)

    def configure_scheduler(self, optimizer):
        """
        配置学习率衰减策略
        
        参数:
            optimizer: 优化器(通过configure_optimizer配置得到的)
        
        返回:
            scheduler: 学习率策略实例或list
        """
        
        return self.step_lr(optimizer, step_size=30, gamma=0.1)
    
    ##########################################################################
    ####### Trainer: Train ######
    ##########################################################################
    def train_dataloader(self):
        """
        训练数据集批量控制加载器, 可以设置批量的大小, 是否对数据进行洗牌(shuffle)等
        
        返回:
            DataLoader: 数据加载器
        """
        return self.get_dataloader(
            phase='train',  # [M] 训练DataLoader
            input_size=28,  # [O] 输入到模型的图片大小
            batch_size=32,  # [O] 输入到模型的最大批量数
            # data_augment=[
            #     self.random_brightness(factor=0.3),
            #     self.random_rotation(degrees=30)
            # ], # [O] 数据增强
            random_order=False, # [O] 数据增强变换方法顺序是否随机
            normalize=True,     # [O] 是否对输入的数据进行归一化
            drop_last=False,    # [O] 一次epoch中最后一次批量数可能不足, 是否丢弃
            shuffle=False)      # [O] 加载的数据是否随机洗牌
     
    def training_step(self, batch, batch_idx):
        """
        训练过程中, 迭代一次batch数据, 就会触发一次training_step的调用,训练,统计metrics
        
        参数:
            batch: 一个batch的数据内容, 一般包括图片(image), 图片标签(labels), 图片路径(path).
                具体batch中内容受prepare_dataset接口的实现会有所不同
            batch_idx: 本轮epoch批量迭代次数
            
        返回:
            metrics: 必须包含loss关键字, log(日志模块)和progress_bar(进度条显示)是可选的
        """
        x, y, _ = batch
        y_hat = self(x)
        loss = self.cross_entropy(y_hat, y, reduction='mean') # 损失方法
        with torch.no_grad():
            accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean() # 计算争取率
        return {'loss': loss, 'acc': accuracy}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
        log = {'train_loss': avg_loss, 'train_acc': avg_acc}
        return {'progress_bar': log}

    ##########################################################################
    ####### Trainer: validation ######
    ##########################################################################
    def val_dataloader(self):
        """
        同train_dataloader
        """
        return self.get_dataloader(
            phase='val',
            input_size=28,
            batch_size=32,
            normalize=True,
            drop_last=False,
            shuffle=False)
    
    
    def validation_step(self, batch, batch_idx):
        """
        同train_step
        """
        x, y, _ = batch
        y_hat = self(x)
        loss = self.cross_entropy(y_hat, y, reduction='mean')
        accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        return {'loss': loss, 'acc': accuracy}
        
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
        return {'progress_bar': {'val_loss': avg_loss, 'val_acc': avg_acc}}
    
    ##########################################################################
    ####### Trainer: test ######
    ##########################################################################
    def test_dataloader(self):
        return self.get_dataloader(
            phase='test',
            input_size=28,
            batch_size=32,
            random_order=False,
            normalize=True,
            drop_last=False,
            shuffle=False)

    def test_step(self, batch, batch_idx):
        x, y, p = batch
        y_hat = self(x)
        accuracy = (torch.argmax(y_hat, axis=1) == y).float().mean()
        return {'acc': accuracy}
        
    def test_epoch_end(self, outputs):
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
        return {'test_acc': avg_acc}
    
    
trainer = EasyaiTrainer(
    max_epochs=10, # 训练过程遍历完整数据集的总次数(epoch)
    resume=False,  # True: 模型继续上次训练(模型必须没有改变)
    log_rate=2,    # 日志打印的频率, 单位是迭代次数(iteration step) 
    model_summary='full', # 打印模型顶层Memory信息
    model_ckpt={'monitor': 'val_loss', 'period': 2, 'mode': 'min'},
    early_stop={'monitor': 'val_acc', 'patience': 3, 'mode': 'max'}
)

model = CustomClassifier()

# 训练
trainer.fit(model)

# 评估
trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
{'label_names': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'mean': [0.1362, 0.1362, 0.1362],
 'num_classes': 10,
 'num_records': 10000,
 'std': [0.2893, 0.2893, 0.2893]}

--------------------------------------------------------------------------------

   | Name                        | Type              | Params
-------------------------------------------------------------------
0  | model                       | ResNet            | 11 M  
1  | model.conv1                 | Conv2d            | 9 K   
2  | model.bn1                   | BatchNorm2d       | 128   
3  | model.relu                  | ReLU              | 0     
4  | model.maxpool               | MaxPool2d         | 0     
5  | model.layer1                | Sequential        | 147 K 
6  | model.layer1.0              | BasicBlock        | 73 K  
7  | model.layer1.0.conv1        | Conv2d            | 36 K  
8  | model.layer1.0.bn1       

Saving latest checkpoint..
Epoch 00008: early stopping triggered.



(fit)	GPU-0 memory allocated: 259.2 MB	 max memory allocated: 1458.63 MB
--------------------------------------------------------------------------------
{'label_names': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'mean': [0.1362, 0.1362, 0.1362],
 'num_classes': 10,
 'num_records': 10000,
 'std': [0.2893, 0.2893, 0.2893]}



(fit)	GPU-0 memory allocated: 302.28 MB	 max memory allocated: 1458.63 MB
(test)	GPU-0 memory allocated: 302.28 MB	 max memory allocated: 1458.63 MB
--------------------------------------------------------------------------------
{'test_acc': 0.9856151342391968}
--------------------------------------------------------------------------------
